# Introduction
The purpose of this notebook is to compare the results of running Olivo's GRC tools and my code to call the same loci from Pf 6.0 VCF.

See 20161117_run_Olivo_GRC.ipynb for details of running Olivo's GRC code on the Pf6.0 release and 20161118_GRC_from_VCF.ipynb for VCF code.

In [1]:
%run _standard_imports.ipynb

python 3.4.5 |Anaconda 2.2.0 (64-bit)| (default, Jul  2 2016, 17:47:47) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.1
scipy 0.18.0
pandas 0.19.0
numexpr 2.6.1
pysam 0.9.1.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [3]:
output_dir = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161118_compare_Olivo_vs_VCF_GRC_Pf6"
!mkdir -p {output_dir}
reads_6_0_results_fn = "/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161117_run_Olivo_GRC/grc/AllCallsBySample.tab"
vcf_6_0_results_fn = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161118_GRC_from_VCF/Pf_6_GRC_from_vcf.xlsx"

all_calls_crosstab_fn = "%s/all_calls_crosstab.xlsx" % output_dir
discordant_calls_crosstab_fn = "%s/discordant_calls_crosstab.xlsx" % output_dir
discordant_nonmissing_calls_crosstab_fn = "%s/discordant_nonmissing_calls_crosstab.xlsx" % output_dir


In [4]:
tbl_read_results = (
    etl
    .fromtsv(reads_6_0_results_fn)
    .distinct('Sample')
    .rename('mdr2_484[P]', 'mdr2_484[T]')
    .rename('fd_193[P]', 'fd_193[D]')
)
print(len(tbl_read_results.data()))
tbl_read_results

8247


Num,Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
799,FP0008-C,"CVIET,CVMNK",D,H,A,L,I,S,E,N,T,T,"T,I","I,R","N,I","C,R","S,N",I,"A,S","A,G",K,A,A,N,"F,Y",D,V,T,D,E
5706,FP0009-C,CVIET,D,H,A,L,I,S,E,N,T,T,T,I,I,R,N,I,A,A,K,A,A,Y,F,"Y,D",V,T,D,E
6537,FP0015-C,CVIET,D,H,A,L,I,S,E,N,T,T,I,I,I,R,N,I,"A,S","A,G",K,A,A,N,"F,Y",D,V,T,D,E
3112,FP0016-C,CVIET,D,h,A,l,i,s,E,n,t,t,i,I,N,C,S,I,S,G,K,A,A,N,F,D,-,T,D,E
5136,FP0017-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,S,I,"A,F",A,K,A,"A,S",N,Y,D,V,T,D,E


In [5]:
tbl_vcf_results = (
    etl
    .fromxlsx(vcf_6_0_results_fn)
)
print(len(tbl_vcf_results.data()))
tbl_vcf_results

7182


Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
FP0008-C,"CVMNK,CVIET",D,H,A,L,I,"A,S",E,N,T,T,"I,T","R,I",N,C,"S,N",I,"S,A","G,A",K,A,A,N,"Y,F",D,V,T,D,E
FP0009-C,CVIET,D,H,A,L,I,S,E,N,T,T,T,I,I,R,N,I,A,A,K,A,A,Y,F,Y,V,T,D,E
FP0015-C,CVIET,D,H,A,L,I,S,E,N,T,T,I,I,I,R,N,I,"S,A","G,A",K,A,A,N,"Y,F",D,V,T,D,E
FP0016-C,"CVMNK,CVIET",D,H,A,L,I,S,E,N,T,T,I,"R,I",N,C,S,I,S,G,K,A,A,N,F,D,V,T,D,E
FP0017-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,S,I,"F,A",A,K,A,"A,S",N,Y,D,V,T,D,E


In [6]:
tbl_both_results = (
    tbl_read_results
    .cutout('Num')
    .join(tbl_vcf_results.replaceall('.', '-'), key='Sample', lprefix='reads_', rprefix='vcf_')
    .convertall(lambda x: ','.join(sorted(x.split(','))))
)
print(len(tbl_both_results.data()))


7182


In [7]:
df_both_results = tbl_both_results.todataframe()

In [8]:
loci = list(tbl_read_results.header()[2:])
print(len(loci))

29


In [9]:
writer = pd.ExcelWriter(all_calls_crosstab_fn)
for i in np.arange(1, 30):
    pd.crosstab(
        df_both_results.ix[:, i],
        df_both_results.ix[:, i+29],
        margins=True
    ).to_excel(writer, loci[i-1].split('[')[0]) # Excel doesn't like the [CMNVK] like endings
writer.save()

In [10]:
writer = pd.ExcelWriter(discordant_calls_crosstab_fn)
for i in np.arange(1, 30):
    pd.crosstab(
        df_both_results.ix[:, i][df_both_results.ix[:, i].str.upper() != df_both_results.ix[:, i+29].str.upper()],
        df_both_results.ix[:, i+29][df_both_results.ix[:, i].str.upper() != df_both_results.ix[:, i+29].str.upper()],
        margins=True
    ).to_excel(writer, loci[i-1].split('[')[0])
writer.save()

In [11]:
writer = pd.ExcelWriter(discordant_nonmissing_calls_crosstab_fn)
for i in np.arange(1, 30):
    pd.crosstab(
        df_both_results.ix[:, i][
            (df_both_results.ix[:, i].str.upper() != df_both_results.ix[:, i+29].str.upper()) &
            (df_both_results.ix[:, i] != '-') &
            (df_both_results.ix[:, i+29] != '-')
        ],
        df_both_results.ix[:, i+29][
            (df_both_results.ix[:, i].str.upper() != df_both_results.ix[:, i+29].str.upper()) &
            (df_both_results.ix[:, i] != '-') &
            (df_both_results.ix[:, i+29] != '-')
        ],
        margins=True
    ).to_excel(writer, loci[i-1].split('[')[0])
writer.save()

## Different allele discordances reads vs vcf

Manual inspection of discordant_nonmissing_calls_crosstab_fn showed the following 8 discordant alleles (ignoring all het vs hom discordances)

- crt_72-76[CVMNK]: CVIDT,CVIET vs CVIDK,CVIET
- crt_72-76[CVMNK]: CVIET,CVMNK,SVMNT vs CVMNK,CVMNT
- dhps_436[S]: A,F vs A,S
- dhps_540[K]: E,N vs E,K (5)

The following few cells show which samples were discordant at the above. See 20161118_GRC_from_VCF.ipynb
for details of VCF calls at these. Comments below refer to manual inspection of these calls

In [12]:
# Het call made at final SNP despite only one read
# Unclear which result really correct
df_both_results[
    (df_both_results['reads_crt_72-76[CVMNK]'] == 'CVIDT,CVIET') &
    (df_both_results['vcf_crt_72-76[CVMNK]'] == 'CVIDK,CVIET')]

,reads_Sample,reads_crt_72-76[CVMNK],reads_crt_24[D],reads_crt_97[H],reads_crt_144[A],reads_crt_148[L],reads_crt_194[I],reads_crt_220[A],reads_crt_271[Q],reads_crt_326[N],...,vcf_dhps_540[K],vcf_dhps_581[A],vcf_dhps_613[A],vcf_mdr1_86[N],vcf_mdr1_184[Y],vcf_mdr1_1246[D],vcf_arps10_127[V],vcf_mdr2_484[T],vcf_fd_193[D],vcf_exo_415[E]
5646,PV0174-C,"CVIDT,CVIET",D,H,"A,F","i,l",t,-,E,N,...,K,A,A,N,Y,D,"M,V","I,T",D,E


In [13]:
# Many variants here had small numbers of ALT reads but were called het
# Reads results probably correct
df_both_results[
    (df_both_results['reads_crt_72-76[CVMNK]'] == 'CVIET,CVMNK,SVMNT') &
    (df_both_results['vcf_crt_72-76[CVMNK]'] == 'CVMNK,CVMNT')]

,reads_Sample,reads_crt_72-76[CVMNK],reads_crt_24[D],reads_crt_97[H],reads_crt_144[A],reads_crt_148[L],reads_crt_194[I],reads_crt_220[A],reads_crt_271[Q],reads_crt_326[N],...,vcf_dhps_540[K],vcf_dhps_581[A],vcf_dhps_613[A],vcf_mdr1_86[N],vcf_mdr1_184[Y],vcf_mdr1_1246[D],vcf_arps10_127[V],vcf_mdr2_484[T],vcf_fd_193[D],vcf_exo_415[E]
6420,QG0211-C,"CVIET,CVMNK,SVMNT",D,H,A,L,I,A,Q,N,...,K,A,A,Y,Y,D,V,T,D,E


In [14]:
# Get wrong call here becuase middle GT is 0/0. Would get correct call if we used PGT call which is 1|0
# Reads results probably correct
df_both_results[(df_both_results['reads_dhps_436[S]'] == 'A,F') & (df_both_results['vcf_dhps_436[S]'] == 'A,S')]

,reads_Sample,reads_crt_72-76[CVMNK],reads_crt_24[D],reads_crt_97[H],reads_crt_144[A],reads_crt_148[L],reads_crt_194[I],reads_crt_220[A],reads_crt_271[Q],reads_crt_326[N],...,vcf_dhps_540[K],vcf_dhps_581[A],vcf_dhps_613[A],vcf_mdr1_86[N],vcf_mdr1_184[Y],vcf_mdr1_1246[D],vcf_arps10_127[V],vcf_mdr2_484[T],vcf_fd_193[D],vcf_exo_415[E]
3888,PH0919-C,CVIET,D,H,A,L,I,S,E,"N,S",...,E,A,A,N,"F,Y",D,V,I,"D,Y",E


In [101]:
# For all of the following, final variant looks like a het (always 2+ reads) but called hom
# Reads results probably correct for all of these
df_both_results[(df_both_results['reads_dhps_540[K]'] == 'E,N') & (df_both_results['vcf_dhps_540[K]'] == 'E,K')]

,reads_Sample,reads_crt_72-76[CVMNK],reads_crt_24[D],reads_crt_97[H],reads_crt_144[A],reads_crt_148[L],reads_crt_194[I],reads_crt_220[A],reads_crt_271[Q],reads_crt_326[N],...,vcf_dhps_540[K],vcf_dhps_581[A],vcf_dhps_613[A],vcf_mdr1_86[N],vcf_mdr1_184[Y],vcf_mdr1_1246[D],vcf_arps10_127[V],vcf_mdr2_484[T],vcf_fd_193[D],vcf_exo_415[E]
701,PD0009-01,CVIET,D,H,A,L,I,S,E,S,...,"E,K",G,A,N,Y,D,M,Y,V,E
3578,PH0566-C,CVIET,D,H,A,L,I,S,E,S,...,"E,K",A,A,N,F,D,M,Y,V,G
3701,PH0718-C,CVIET,D,H,A,L,i,s,E,S,...,"E,K",A,A,N,Y,D,M,Y,V,E
3874,PH0905-C,CVIET,D,"H,L",A,L,I,S,e,S,...,"E,K",A,A,N,F,D,"M,V",Y,V,"E,G"
4231,PH1422-C,CVIET,D,H,A,L,I,s,E,s,...,"E,K",A,A,N,F,D,M,Y,V,E


# Conclusion
I have written code to call the same set of variants as Olivo's read-based analysis. This uses the same input file (grc.properties) so can easily be applied to future call sets and loci. The code to create calls from the vcf is very quick, with all loci called in all samples in about 2 minutes.

The results were highly concordant. There were no homozygous call differences. There were 8 cases where different alleles were called as a part of a het call. I've manually checked all these, and in 7 cases I think the calls in the vcf are wrong, for example two different variants have what look from read counts like het calls, but one is called as het and one as hom. The eighth sample with a different allele had only one alternate read which VCF called as het but Olivo's code as hom - so not really clear what truth is here. Other than that the only differences are het vs hom calls (and the few I've checked looked like borderline calls), or called vs missing. I inspected the reads for three sample where VCF had hom calls at crt_97, whereas Olivo's code had het calls. In each sample, there were clearly good quality reads and bases with the minor allele, but these weren't reflected in the VCF, for reasons I don't understand. Olivo's read-based calls have slightly higher missingness, but this is as expected as I have done no filtering on the vcf-based calls.

The VCF-based calls can be found in /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161118_GRC_from_VCF/Pf_6_GRC_from_vcf.xlsx. Missed calls are denoted by ".". Frameshift mutations are denoted by "!". Samples that have two or more het calls at the locus which are unphased are denoted by 'X'. Samples which have two or more phase groups at the locus are denoted by "?".

Calls are only set to missing if all genotypes at locus missing. If only some missing, have assumed hom ref (this is important due to pecularities at crt_72-76, where genotypes between insertion at 403618 and deletion at 403622 are set to missing in samples that have the insertion/deletion.

With the VCF-based code, two samples appeared to have frameshift mutations in crt_72-76. Manual inspection showed borderline het/hom calls. PA0490-C has hom alt call (4 ref, 134 alt reads) at 403618 insertion and het call (5 ref, 137 alt reads) at 403622 deletion. QV0090-C has het call (3 ref, 104 alt reads) at 403618 insertion and hom alt call (3 ref, 106 alt reads) at 403622 deletion. For all these calls, the genotype quality is low (GQ < 30) reflecting similar likelihoods of het and hom alt calls.

The following loci had samples with unphased hets (number in brackets are numbers of samples): crt_72-76 (6), crt_333 (4), dhps_436 (17), dhps_540 (28). A further five samples had hets in different phase groups at crt_72-76. All of these samples could not be called by VCF-based code.

Of all the variants discovered in these loci, only two (at Pf3D7_07_v3:404836 and Pf3D7_07_v3:405559) failed variant filters. In each case there was only one sample with a heterozygous call at the variant, and the position was the last in the codon and the different alleles made no change in the amino acid.

In summary, the calls from the vcf and from the reads are very similar, and the problems of calling CRT seem to be a thing of the past. Having said that, in the few cases where there was a discrepancy between calls, in general the reads-based calls are right. There were also a few cases that couldn't be called from the VCF, e.g. due to unresolved phasing, and two cases of the VCF incorrectly predicting a frameshift. Therefore I would recommend using Olivo's reads-based calls code for the GRC.

